In [27]:
import pint
import pyproj
import projnames
import geopandas as gpd
import pandas as pd
import numpy as np
import pint_pandas
ureg = pint.get_application_registry() #pint.UnitRegistry()

In [28]:
epsgs = list(projnames.by_epsg.keys()) + [4326]

In [29]:
for epsg in epsgs:
    ureg.define('epsg_%s = [epsg_%s]; offset: 1 = _' % (epsg, epsg))

In [30]:
ureg.define('@alias epsg_4326 = latlon')

In [31]:
class C(object):
    def __init__(self, x, y):
        self.x = x
        self.y = y
    def __sub__(self, other):
        if not isinstance(other, C): return self
        return C(self.x - other.x, self.y - other.y)
    def __mul__(self, other):
        return C(self.x * other, self.y * other)
    def __truediv__(self, other):
        return C(self.x / other, self.y / other)
    def __add__(self, other):
        if not isinstance(other, C): return self
        return C(self.x + other.x, self.y + other.y)
    def __format__(self, format_spec):
        return str(self.x) + ";" + str(self.y)
    def __repr__(self):
        return str(self.x) + ";" + str(self.y)
    

In [32]:
def register_epsg(epsg, c):
    def tolatlon(ureg, x):
        return ureg.Quantity(
            C(*pyproj.Transformer.from_crs(
                         epsg, 4326, always_xy=True).transform(x.m.x, x.m.y)), "latlon")
    def fromlatlon(ureg, x):
        return ureg.Quantity(
            C(*pyproj.Transformer.from_crs(
                         4326, epsg, always_xy=True).transform(x.m.x, x.m.y)), "epsg_%s" % epsg)

    c.add_transformation("epsg_%s" % epsg, 'latlon', tolatlon)
    c.add_transformation("latlon", "epsg_%s" % epsg, fromlatlon)

    if (epsg in projnames.by_epsg) and ("UTM" in projnames.by_epsg[epsg]):
        c.add_transformation("delta_epsg_%s" % epsg, 'm',
                             lambda ureg, x: ureg.Quantity((x.m.x**2+x.m.y**2)**0.5, "m"))

c = pint.Context('maps')
for epsg in epsgs:
    register_epsg(epsg, c)

ureg.add_context(c)

In [33]:
p1 = ureg.Quantity(C(15, 2), "epsg_32611")
p2 = ureg.Quantity(C(2, 1), "epsg_32611")
d = p1 - p2

In [34]:
p1.to("latlon", "maps")

-121.48860949942477;1.8038754962662663e-05 <Unit('epsg_4326')>

In [35]:
d.to("m", "maps")

13.038404810405298 <Unit('meter')>

In [36]:
p3 = ureg.Quantity(C(72, 10), "latlon")
p4 = ureg.Quantity(C(73, 9), "latlon")

In [37]:
(p4 - p3).to("m", "maps")

20719663.977746077 <Unit('meter')>

In [38]:
df = pd.DataFrame({
     "p": pd.Series([p1, p2], dtype=pint_pandas.pint_array.PintType(p1.u))
})

In [44]:
df

,p
0,15;2
1,2;1


In [41]:
df.p.loc[0].to("latlon", "maps")

-121.48860949942477;1.8038754962662663e-05 <Unit('epsg_4326')>

In [45]:
df = pd.DataFrame({
     "p": pd.Series([1, 2, 3], dtype="pint[m]")
})

In [50]:
df.p.values._data.dtype

dtype('int64')